In [10]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta, timezone

# --- 常數定義 ---
DAYS_PER_YEAR = 365.25
SECONDS_PER_DAY = 24 * 3600

# --- Binance K線資料抓取函數 ---
def get_binance_kline(symbol: str, interval: str, end_time: datetime, limit: int = 300) -> pd.DataFrame:
    """
    從 Binance API 取得歷史K線資料
    """
    base_url = "https://api.binance.com/api/v3/klines"
    end_timestamp = int(end_time.timestamp() * 1000)
    params = {
        "symbol": symbol.upper(),
        "interval": interval,
        "endTime": end_timestamp,
        "limit": limit
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    df = pd.DataFrame(data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
    return df[["timestamp", "open", "high", "low", "close"]]

# --- 核心策略計算函數 ---
def calculate_optimal_market_making_params(
    asset: str,
    mid_price: float,
    daily_volatility_pct: float,
    target_order_fill_prob: float,
    order_refresh_time_sec: int,
    stop_loss_risk_prob: float,
    max_holding_time_days: float = 30.0,
    profit_factor: float = 2.0
) -> dict:
    """
    根據 GBM 波動率模型，計算最優的合約造市策略參數。
    """
    daily_volatility = daily_volatility_pct / 100.0
    annual_volatility = daily_volatility * np.sqrt(DAYS_PER_YEAR)
    dt_order = order_refresh_time_sec / (DAYS_PER_YEAR * SECONDS_PER_DAY)
    dt_loss = max_holding_time_days / DAYS_PER_YEAR

    # 基礎掛單價差
    p_half_order = target_order_fill_prob / 2.0
    Z_order = norm.ppf(p_half_order)
    base_spread_pct = (annual_volatility * np.sqrt(dt_order) * np.abs(Z_order)) * 100

    # 止盈、止損
    profit_taking_spread_pct = base_spread_pct * profit_factor
    p_half_loss = stop_loss_risk_prob / 2.0
    Z_loss = norm.ppf(p_half_loss)
    stop_loss_spread_pct = (annual_volatility * np.sqrt(dt_loss) * np.abs(Z_loss)) * 100

    return {
        "asset": asset,
        "current_mid_price": mid_price,
        "order_refresh_time_sec": order_refresh_time_sec,
        "bid_spread": round(base_spread_pct, 4),
        "ask_spread": round(base_spread_pct, 4),
        "long_profit_taking_spread": round(profit_taking_spread_pct, 4),
        "short_profit_taking_spread": round(profit_taking_spread_pct, 4),
        "stop_loss_spread": round(stop_loss_spread_pct, 4),
        "Z_score_order": round(np.abs(Z_order), 4),
        "Z_score_stop_loss": round(np.abs(Z_loss), 4)
    }

# --- 自動化流程：從 Binance 抓資料 + 計算 ---
def calculate_from_binance(symbol: str = "XRPUSDT", interval: str = "1h"):
    """
    從 Binance 取得最近 30 天資料，自動計算日化波動率與造市參數。
    """
    end_time = datetime.now(timezone.utc)
    df = get_binance_kline(symbol, interval, end_time, limit=720)  # 約30天的1小時K線

    # 計算對數報酬率
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))
    hourly_vol = df["log_return"].std()

    # 換算成日化波動率
    daily_vol = hourly_vol * np.sqrt(24)
    print(f"📊 {symbol} 日化波動率估計值: {daily_vol*100:.2f}%")

    # 帶入策略參數
    params = {
        "asset": symbol,
        "mid_price": df["close"].iloc[-1],
        "daily_volatility_pct": daily_vol * 100,
        "target_order_fill_prob": 0.25,
        "order_refresh_time_sec": 15,
        "stop_loss_risk_prob": 0.01,
        "max_holding_time_days": 1,
        "profit_factor": 2.5
    }

    result = calculate_optimal_market_making_params(**params)

    print("\n--- 🔬 最優造市參數計算結果 ---")
    for key, value in result.items():
        if "_spread" in key or "time_sec" in key:
            unit = "%" if "spread" in key else "秒"
            print(f"{key:<30}: {value:>.4f} {unit}")
        elif key == "current_mid_price":
            print(f"{key:<30}: {value:>.4f} USDT")
        elif key == "asset":
            print(f"{key:<30}: {value}")

    return result


# --- 主程式執行 ---
if __name__ == "__main__":
    calculate_from_binance("XRPUSDT", interval="1m")


📊 XRPUSDT 日化波動率估計值: 0.50%

--- 🔬 最優造市參數計算結果 ---
asset                         : XRPUSDT
current_mid_price             : 2.1905 USDT
order_refresh_time_sec        : 15.0000 秒
bid_spread                    : 0.0075 %
ask_spread                    : 0.0075 %
long_profit_taking_spread     : 0.0188 %
short_profit_taking_spread    : 0.0188 %
stop_loss_spread              : 1.2796 %
